In [191]:
from google import genai
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# response = client.models.generate_content(
#     model="gemini-2.0-flash", contents="Ge mig en historia om en kanin som bor i skogen på 3 meningar"
# )
# print(response.text)



In [192]:
with duckdb.connect("../../ads_data.duckdb") as con: 
    df_mart = con.execute("SELECT * FROM mart.mart_kultur_media").df()
df_test = df_mart #[df_mart["occupation_field"] == "Kultur, media, design"]
# df_test.to_csv("sample_ads.csv") # måste göra mer finjusteringar! kan inte läsa allas description direkt!


In [267]:
descriptions = df_test["description"]

# top_employer = df_test.groupby("occupation")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
# top_employer = df_test.groupby("employer_name")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
top_employer = df_test.groupby("workplace_city")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()

top_employer
# df_test

,workplace_city,vacancies,description
0,ej angiven,624,"Vi söker nu nya artister, musiker, skådespelar..."
1,Stockholm,166,Vi söker dig som vill bli spindeln i nätet på ...
2,Göteborg,138,Göteborgs universitet möter samhällets utmanin...
3,Malmö,102,"Ref: 20251128 \n\nHälsa-, vård- och omsorgsför..."
4,Boden,74,Research shows that women and other under-repr...
...,...,...,...
93,Örnsköldsvik,1,Känner du att du egentligen vill arbeta med nå...
94,Östersund,1,Vill du bli vår framtida Visuell Merchandiser?...
95,Österåker,1,Internationella Engelska Skolan (IES) is a lea...
96,Östra göinge,1,Östra Göinge kommun med drygt 14 000 invånare ...


In [62]:
top_employer = df_test.groupby("occupation")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
# nr_1 = top_employer[top_employer["employer_name"] == 'Exakta Photo AB']
# nr_1 = top_employer.iloc[0]["description"]
# top_employer["description"][0]
# nr_1 = top_employer[["workplace_city","description"]].iloc[2]
# nr_1.to_csv("for_gemini.txt")
# nr_1.to_csv("for_gemini.csv")
top_employer.head(20)

,occupation,vacancies,description
0,Tolk,110,Vill du ha ett meningsfullt arbete som ger dig...
1,Bibliotekarie,57,I Knivsta går framtidsandan att ta på. Läget m...
2,Layoutare,42,Vi söker medarbetare till vårt huvudkontor i M...
3,Fotograf,41,Vi söker dig som vill utvecklas som bostadsfot...
4,Skolbibliotekarie,37,Vad vi gör och vill\nTyresö gymnasium är inne ...
5,Skådespelare,27,Teater Scarlet har spelat Mordgåtor sedan 2017...
6,Technical Operation Manager/TOM,25,HCLTech is a globally recognized leader in the...
7,Journalist/Reporter,19,Norrköpings Tidningar är mitt uppe i en stark ...
8,"Projektledare: kultur, media, film, dataspel",17,Om myndigheten\nForum för levande historia är ...
9,Arkivarie,16,Har du ett starkt samhällsengagemang och intre...


In [ ]:
# nr_1 = top_employer["employer_name"]
# nr_1["description"].to_csv("top_employer.csv")
# nr_1["description"]
# nr_1

occupation                                                  Tolk
description    Vi söker utbildare inom japanska. \nUppdraget ...
Name: 0, dtype: object

In [68]:
# df_test.reset_index()
nr_1 = top_employer[["occupation","description"]].iloc[7]
nr_1.to_csv("for_gemini.txt")

In [76]:
with open("for_gemini.txt", "r", encoding='utf-8') as file: 
    ad_text = file.read() 
# ad_text = nr_1["description"]
# f"Vad är de vanligaste kraven och meriterna inom {choice_sort}?", 
# Svara på denna fråga: {question} Genom att läsa dessa annonser: 
# f"Vad är de vanligaste arbetsuppgifterna inom {choice_sort}?"
question = "Vad är de vanligaste arbetsuppgifterna inom Göteborg?"
promt = f"""Du är en rekryterare inom Media, Kultur, Design.
Plocka ut max 5 av vanligaste kraven och max av 5 vanligaste meriterade och antal av dessa som efterfrågas i dessa jobbannonser, samt en sammanfattning av krav och meriter som du skriver ut:
{ad_text}

Output ska vara i detta formatet enbart: 

{{
    "sammanfattning": ["sammanfattning"]
    "krav": [erfarenhet1, erfarenhet2, ...]
    "antal krav": [summa erfarenhet1..]
    "meriterande": [meriterande1,meriterande2, ... ]
    "antal meriterande": [summa meriterande1..]
    
}}
"""
# om du inte får ihop samma antal på krav och antal krav balansera ut det som fattas med N/A, 0. Samma med meriterande och antal meriterande. 


In [1]:
max_rows = 100
ad_text_splitted = ad_text.split("\n") #tagit bort alla mellanslag
chunks = [] # lista för textbitar

for i in range(0, len(ad_text_splitted), max_rows): 
    chunk = "\n".join(ad_text_splitted[i:i+max_rows])
    chunks.append(chunk)
    # print(chunk)
    # print("------------------------------------------------------LL_-------------------")
    # return chunks

# print(ad_text_splitted)

# text = ["description"]
# ad_text[:4000]
# .to_csv("rows.txt")


NameError: name 'ad_text' is not defined

In [ ]:
len(chunks)

6

In [ ]:
question = "Vad är de vanligaste arbetsuppgifterna inom Göteborg?"
promt = f"""Du är en rekryterare inom Media, Kultur, Design.
Plocka ut max 5 av vanligaste kraven och max av 5 vanligaste meriterade och antal av dessa som efterfrågas i dessa jobbannonser, samt en sammanfattning av krav och meriter som du skriver ut:
{ad_text}

Output ska vara i detta formatet enbart: 

{{
    "sammanfattning": ["sammanfattning"]
    "krav": [erfarenhet1, erfarenhet2, ...]
    "antal krav": [summa erfarenhet1..]
    "meriterande": [meriterande1,meriterande2, ... ]
    "antal meriterande": [summa meriterande1..]
    
}}
"""

In [ ]:
from google import generativeai as oldGenai
from google.generativeai.types import GenerationConfig

#6 chunks tar 14 sekunder

# config = GenerationConfig(temperature=0.0)

# response = client.models.generate_content(
#     model="gemini-2.0-flash", contents=promt, generation_config={"temperature": 0.0}
# )
response_list = []
for chunk in chunks:
    promt = f"""Du är en rekryterare inom Media, Kultur, Design.
    Plocka ut max 5 av vanligaste kraven och max av 5 vanligaste meriterade och antal av dessa som efterfrågas i dessa jobbannonser, samt en sammanfattning av krav och meriter som du skriver ut:
    {chunk}

    Output ska vara i detta formatet enbart: 

    {{
        "sammanfattning": ["sammanfattning"]
        "krav": [erfarenhet1, erfarenhet2, ...]
        "antal krav": [summa erfarenhet1..]
        "meriterande": [meriterande1,meriterande2, ... ]
        "antal meriterande": [summa meriterande1..]
        
    }}
    """
    model = oldGenai.GenerativeModel(
        model_name="gemini-2.0-flash",
        generation_config=GenerationConfig(
            temperature=0.0,         
            top_p=1.0,  # Säkerställer att sampling inte påverkas   #verkar inte göra skillnad
            top_k=1     # Tar bort variation)                       #verkar inte göra skillnad    
        )
    )
    response = model.generate_content(promt)
    print(response.text)
    response_list.append(response.text)


```json
{
    "sammanfattning": "Dessa jobbannonser söker engagerade och drivna journalister och en nyhetschef med starkt intresse för lokaljournalistik och digital utveckling. Viktiga egenskaper är nyfikenhet, kreativitet, god kommunikationsförmåga och förmågan att arbeta både självständigt och i team. Ledarskapserfarenhet efterfrågas för nyhetschefsrollen.",
    "krav": [
        "Journalistutbildning eller motsvarande arbetslivserfarenhet",
        "Erfarenhet av journalistiskt arbete på en nyhetsredaktion",
        "God stilistisk förmåga och förmåga att knyta kontakter",
        "B-körkort",
        "Erfarenhet som redaktör/nyhetschef på en större redaktion"
    ],
    "antal krav": [
        3,
        3,
        1,
        1,
        1
    ],
    "meriterande": [
        "Erfarenhet av granskande journalistik",
        "Erfarenhet av bildjournalistik",
        "Kunskap om lokalområdet (Sollefteå)",
        "Lokal kännedom (Helsingborg)",
        "Erfarenhet av personalansvar"
  

In [101]:
response_list

['```json\n{\n    "sammanfattning": "Dessa jobbannonser söker engagerade och drivna journalister och en nyhetschef med starkt intresse för lokaljournalistik och digital utveckling. Viktiga egenskaper är nyfikenhet, kreativitet, god kommunikationsförmåga och förmågan att arbeta både självständigt och i team. Ledarskapserfarenhet efterfrågas för nyhetschefsrollen.",\n    "krav": [\n        "Journalistutbildning eller motsvarande arbetslivserfarenhet",\n        "Erfarenhet av journalistiskt arbete på en nyhetsredaktion",\n        "God stilistisk förmåga och förmåga att knyta kontakter",\n        "B-körkort",\n        "Erfarenhet som redaktör/nyhetschef på en större redaktion"\n    ],\n    "antal krav": [\n        3,\n        3,\n        1,\n        1,\n        1\n    ],\n    "meriterande": [\n        "Erfarenhet av granskande journalistik",\n        "Erfarenhet av bildjournalistik",\n        "Kunskap om lokalområdet (Sollefteå)",\n        "Lokal kännedom (Helsingborg)",\n        "Erfarenh

In [102]:
promt2 = f"""kan du analysera detta och summera till en enda dictionary: 
{response_list}

    Output ska vara i detta formatet enbart: 

    {{
        "sammanfattning": ["sammanfattning"]
        "krav": [erfarenhet1, erfarenhet2, ...]
        "antal krav": [summa erfarenhet1..]
        "meriterande": [meriterande1,meriterande2, ... ]
        "antal meriterande": [summa meriterande1..]
        
    }}
    """
model = oldGenai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config=GenerationConfig(
        temperature=0.0,         
        top_p=1.0,  # Säkerställer att sampling inte påverkas   #verkar inte göra skillnad
        top_k=1     # Tar bort variation)                       #verkar inte göra skillnad    
    )
)
response2 = model.generate_content(promt2)
print(response2.text)
# response_list.append(response.text)

```json
{
    "sammanfattning": [
        "Dessa jobbannonser söker engagerade och drivna journalister och en nyhetschef med starkt intresse för lokaljournalistik och digital utveckling. Viktiga egenskaper är nyfikenhet, kreativitet, god kommunikationsförmåga och förmågan att arbeta både självständigt och i team. Ledarskapserfarenhet efterfrågas för nyhetschefsrollen.",
        "Samtliga roller kräver ett starkt intresse för samhällsfrågor och nyhetsarbete. Reporterrollerna betonar erfarenhet av redaktionellt arbete och undersökande journalistik, medan utbildningsprogrammen öppnar dörren för de utan formell journalistisk bakgrund men med engagemang och kommunikationsförmåga.",
        "Sveriges Radio söker engagerade personer till en betald journalistutbildning med start hösten 2025. Utbildningen är skräddarsydd och kombinerar praktik på redaktionen med distansstudier. Fokus ligger på att lära sig journalistiskt hantverk, producera nyhetsinslag och utveckla ljudets kraft för att skapa 

In [104]:
# print(repr(response.text))
import json
data = response2.text
cleaned = data.strip("```json") #.replace("\n", "").strip().replace("  ", "")
cleaned
# data

'\n{\n    "sammanfattning": [\n        "Dessa jobbannonser söker engagerade och drivna journalister och en nyhetschef med starkt intresse för lokaljournalistik och digital utveckling. Viktiga egenskaper är nyfikenhet, kreativitet, god kommunikationsförmåga och förmågan att arbeta både självständigt och i team. Ledarskapserfarenhet efterfrågas för nyhetschefsrollen.",\n        "Samtliga roller kräver ett starkt intresse för samhällsfrågor och nyhetsarbete. Reporterrollerna betonar erfarenhet av redaktionellt arbete och undersökande journalistik, medan utbildningsprogrammen öppnar dörren för de utan formell journalistisk bakgrund men med engagemang och kommunikationsförmåga.",\n        "Sveriges Radio söker engagerade personer till en betald journalistutbildning med start hösten 2025. Utbildningen är skräddarsydd och kombinerar praktik på redaktionen med distansstudier. Fokus ligger på att lära sig journalistiskt hantverk, producera nyhetsinslag och utveckla ljudets kraft för att skapa e

In [105]:
type(cleaned)
import pandas as pd
data = json.loads(cleaned)

df_krav = pd.DataFrame({
    "krav": data["krav"], 
    "antal krav": data["antal krav"]
})
df_krav
# df = pd.DataFrame(data)
# df
# df = pd.DataFrame(df)
# df

,krav,antal krav
0,Journalistutbildning eller motsvarande arbetsl...,3
1,Erfarenhet av journalistiskt arbete på en nyhe...,3
2,God stilistisk förmåga och förmåga att knyta k...,1
3,B-körkort,2
4,Erfarenhet som redaktör/nyhetschef på en störr...,1
5,Relevant utbildning och/eller erfarenhet från ...,2
6,Erfarenhet av redaktionellt nyhetsarbete,1
7,Gedigen digital kompetens,1
8,"Stort intresse av samhällsfrågor, kultur och n...",15
9,Minst gymnasieutbildning,8


In [106]:
df_merit = pd.DataFrame({
    "meriterande": data["meriterande"],
    "antal meriterande": data["antal meriterande"]
})
df_merit

,meriterande,antal meriterande
0,Erfarenhet av granskande journalistik,3
1,Erfarenhet av bildjournalistik,2
2,Kunskap om lokalområdet (Sollefteå),1
3,Lokal kännedom (Helsingborg),1
4,Erfarenhet av personalansvar,1
5,Fackkunskaper,1
6,God förmåga att uttrycka sig i tal och skrift,1
7,God lokalkännedom,1
8,Brett kontaktnät,1
9,Eftergymnasial utbildning,11


In [107]:
sammanfattning = data["sammanfattning"]
sammanfattning

['Dessa jobbannonser söker engagerade och drivna journalister och en nyhetschef med starkt intresse för lokaljournalistik och digital utveckling. Viktiga egenskaper är nyfikenhet, kreativitet, god kommunikationsförmåga och förmågan att arbeta både självständigt och i team. Ledarskapserfarenhet efterfrågas för nyhetschefsrollen.',
 'Samtliga roller kräver ett starkt intresse för samhällsfrågor och nyhetsarbete. Reporterrollerna betonar erfarenhet av redaktionellt arbete och undersökande journalistik, medan utbildningsprogrammen öppnar dörren för de utan formell journalistisk bakgrund men med engagemang och kommunikationsförmåga.',
 'Sveriges Radio söker engagerade personer till en betald journalistutbildning med start hösten 2025. Utbildningen är skräddarsydd och kombinerar praktik på redaktionen med distansstudier. Fokus ligger på att lära sig journalistiskt hantverk, producera nyhetsinslag och utveckla ljudets kraft för att skapa engagerande innehåll. Viktigt är intresse för samhällsf

In [108]:
import plotly.express as px
# px.pie(df, names=["krav", "meriterande"], values=["antal krav", "antal meriterande"])
px.bar(df_krav, x="krav", y="antal krav")


In [109]:
px.bar(df_merit, y="antal meriterande", x="meriterande")